In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import log_loss
from datetime import datetime, timedelta
from scipy import sparse
from scipy.stats import norm
from scipy.special import logit
import warnings
warnings.filterwarnings("ignore")

**У меня вдруг начало умирать ядро при попытке даже просто прочитать датасет, придется работать с урезанной версией**

In [2]:
df = pd.read_csv('../../data/data.csv',
                 usecols=['date_time', 'zone_id', 'banner_id', 'os_id',
                          'country_id', 'banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0',
                          'coeff_sum1', 'clicks'],
                 nrows=3000000)
df.head()

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,clicks
0,2021-09-27 00:01:30.000000,0,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1
1,2021-09-26 22:54:49.000000,1,1,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1
2,2021-09-26 23:57:20.000000,2,2,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1
3,2021-09-27 00:04:30.000000,3,3,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1
4,2021-09-27 00:06:21.000000,4,4,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1


In [3]:
days = df['date_time'].apply(lambda x: x[:10])
days.value_counts()

2021-09-26    570959
2021-09-29    458094
2021-09-27    431322
2021-09-28    423362
2021-10-02    418879
2021-09-30    365230
2021-10-01    332153
2021-09-01         1
Name: date_time, dtype: int64

**Очищаем данные**

In [4]:
df = df[df['banner_id0'] == df['banner_id']]
df

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,clicks
1,2021-09-26 22:54:49.000000,1,1,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1
2,2021-09-26 23:57:20.000000,2,2,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1
3,2021-09-27 00:04:30.000000,3,3,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1
4,2021-09-27 00:06:21.000000,4,4,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1
5,2021-09-27 00:06:50.000000,5,5,2,2,5,0.004,0.337634,-3.222757,37,0.004,0.338195,-3.221755,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2021-10-01 20:48:38.000000,207,384,1,3,384,0.003,0.037216,-3.270952,281,0.003,0.036387,-3.303162,0
2999996,2021-09-26 17:32:58.000000,42,21,0,0,21,0.014,0.011260,-3.667763,49,0.014,0.022230,-3.712493,0
2999997,2021-09-27 11:24:50.000000,0,361,1,0,361,0.010,0.005333,-5.494210,1239,0.046,0.009252,-7.533400,0
2999998,2021-09-30 00:49:17.000000,86,57,1,1,57,0.013,0.030828,-3.825201,169,0.013,0.034020,-3.845763,0


**Добавляем те же фичи, что и в первом дз**

In [5]:
def get_features(df):
    # время
    hour = df['date_time'].apply(lambda x: int(x[11:13]))
    df['hour'] = hour
    df['daytime'] = hour // 6

    # заменяем разные редкие категории на общую
    zone_mask = df['zone_id'].isin(df.groupby('zone_id')['date_time'].count().sort_values(key=lambda x: -x).index[250:])
    df.loc[zone_mask, 'zone_id'] = -1

    banner_mask = df['banner_id'].isin(df.groupby('banner_id')['date_time'].count().sort_values(key=lambda x: -x).index[600:])
    df.loc[banner_mask, 'banner_id'] = -1

    os_mask = df['os_id'].isin(df.groupby('os_id')['date_time'].count().sort_values(key=lambda x: -x).index[7:])
    df.loc[os_mask, 'os_id'] = -1


    # добавляем интеракции - зона + операционка, баннер + час, баннер + время суток, баннер + операционка (много выходит, да)

    df['zone+os'] = df['zone_id'].astype(str) + '+' + df['os_id'].astype(str)
    df['hour+banner'] = df['hour'].astype(str) + '+' + df['banner_id'].astype(str)
    df['daytime+banner'] = df['daytime'].astype(str) + '+' + df['banner_id'].astype(str)
    df['os+banner'] = df['os_id'].astype(str) + '+' + df['banner_id'].astype(str)

In [6]:
get_features(df)
df.to_csv('df_features.csv', index=False)
df

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,clicks,hour,daytime,zone+os,hour+banner,daytime+banner,os+banner
1,2021-09-26 22:54:49.000000,1,1,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,22,3,1+0,22+1,3+1,0+1
2,2021-09-26 23:57:20.000000,2,2,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,23,3,2+0,23+2,3+2,0+2
3,2021-09-27 00:04:30.000000,3,3,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,0,0,3+1,0+3,0+3,1+3
4,2021-09-27 00:06:21.000000,4,4,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,0,0,4+1,0+4,0+4,1+4
5,2021-09-27 00:06:50.000000,5,5,2,2,5,0.004,0.337634,-3.222757,37,0.004,0.338195,-3.221755,1,0,0,5+2,0+5,0+5,2+5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2021-10-01 20:48:38.000000,207,384,1,3,384,0.003,0.037216,-3.270952,281,0.003,0.036387,-3.303162,0,20,3,207+1,20+384,3+384,1+384
2999996,2021-09-26 17:32:58.000000,42,21,0,0,21,0.014,0.011260,-3.667763,49,0.014,0.022230,-3.712493,0,17,2,42+0,17+21,2+21,0+21
2999997,2021-09-27 11:24:50.000000,0,361,1,0,361,0.010,0.005333,-5.494210,1239,0.046,0.009252,-7.533400,0,11,1,0+1,11+361,1+361,1+361
2999998,2021-09-30 00:49:17.000000,86,57,1,1,57,0.013,0.030828,-3.825201,169,0.013,0.034020,-3.845763,0,0,0,86+1,0+57,0+57,1+57


In [3]:
df = pd.read_csv('df_features.csv')

**Готовимся к обучению модели**

In [4]:
target = df['clicks']
df.drop('clicks', axis=1, inplace=True)

In [5]:
def get_sparse(df_train, df_val, df_test):
    ohe = OneHotEncoder(handle_unknown='ignore')
    scaler = StandardScaler()
    features = ['zone_id', 'banner_id', 'os_id', 'country_id', 'hour', 'daytime',
                'zone+os', 'hour+banner', 'daytime+banner', 'os+banner']
    to_train = []
    to_val = []
    to_test = []
    for feature in features:
        to_train.append(ohe.fit_transform(df_train[[feature]]))
        to_val.append(ohe.transform(df_val[[feature]]))
        to_test.append(ohe.transform(df_test[[feature]]))


    X_train = sparse.hstack(to_train)
    X_val = sparse.hstack(to_val)
    X_test = sparse.hstack(to_test)


    X_train = X_train.tocsr()
    X_val = X_val.tocsr()
    X_test = X_test.tocsr()
    return X_train, X_val, X_test

In [6]:
df_train, y_train = df[df['date_time'] < '2021-10-01'], target[df['date_time'] < '2021-10-01']
df_val, y_val = df[(df['date_time'] >= '2021-10-01') & (df['date_time'] < '2021-10-02')], target[(df['date_time'] >= '2021-10-01') & (df['date_time'] < '2021-10-02')]
df_val_new = df_val.copy()
df_val_new['banner_id'] = df_val_new['banner_id1']
df_test, y_test = df[df['date_time'] > '2021-10-02'], target[df['date_time'] > '2021-10-02']  
df_test_new = df_test.copy()
df_test_new['banner_id'] = df_test_new['banner_id1']

In [7]:
X_train, X_val, X_test = get_sparse(df_train, df_val, df_test)
_, X_val_new, X_test_new = get_sparse(df_train, df_val_new, df_test_new)

In [8]:
def create_model(C, X_train, y_train):
    clf = LogisticRegression(C=C, solver='liblinear')
    clf.fit(X_train, y_train)
    return clf

**Считаем СIPS**

In [9]:
def normal1_greater_than_normal2(mu1, sigma1, mu2, sigma2):
    mu_diff = mu1 - mu2
    sigma_diff = np.sqrt(sigma1 ** 2 + sigma2 ** 2)
    return 1 - norm.cdf(0, loc=mu_diff, scale=sigma_diff)

def compare_normals(m1, s1, m2, s2):
    m = m1 - m2
    s = np.sqrt(s1 ** 2 + s2 ** 2)
    res = 1 - norm.cdf(x=0, loc=m, scale=s)
    return res


Lambda = 10

for C in [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]:
    model = create_model(C, X_train, y_train)
    
    pi0 = compare_normals(df_val['coeff_sum0'], df_val['g0'], df_val['coeff_sum1'], df_val['g1'])
    
    coeff_sum0_new = logit(model.predict_proba(X_val)[:, 1])
    coeff_sum1_new = logit(model.predict_proba(X_val_new)[:, 1])
    pi1 = compare_normals(coeff_sum0_new, df_val['g0'], coeff_sum1_new, df_val['g1']) 
    
    CIPS = (y_val.values * np.minimum(Lambda, np.nan_to_num(pi1 / pi0))).sum() / len(y_val)
    y_pred = model.predict_proba(X_val)[:, 1]
    score = log_loss(y_val, y_pred)
    print(C, round(CIPS, 3), round(score, 3))

0.001 0.454 0.435
0.005 0.463 0.421
0.01 0.465 0.418
0.05 0.465 0.415
0.1 0.463 0.415
0.5 0.451 0.416
1 0.446 0.418
5 0.429 0.421
10 0.42 0.423
50 0.414 0.427
100 0.412 0.429


**CIPS и logloss более или менее согласны друг с другом. Лучшее С по logloss = 0.05 или 0.1 (у меня и в прошлый раз тоже так вышло), по CIPS - 0.01 или 0.05. Посмотрим на 0.05 :)**

In [11]:
model05 = create_model(0.05, X_train, y_train)
y_pred = model05.predict_proba(X_test)[:, 1]
score = log_loss(y_test, y_pred)
print(score)

0.3936262804940591


**Логлосс ужасный в любом случае... вероятно, дело в урезанной выборке - баннеры этого не прощают**